# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [2]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [2]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001L8cQMAS'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [3]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [6]:
display(emails_df)

,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!"
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insight would be appreciated!"
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 Infinity Scarves please. My wife loves collecting scarves in different colors and patterns."
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the description mentions it can be worn as a lightweight blanket. At $22, is the material good enough quality to use as a lap blanket? Or is it more like a thick wrapping scarf? I'm considering buying it as a gift for my grandmother. Thank you!"
5,E006,NaN,"Hey there, I was thinking of ordering a pair of CBT8901 Chelsea Boots, but I'll wait until Fall to actually place the order. My name is Sam and I need some new boots for the colder months."
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cable Knit Beanies and 2 pairs of FZZ1098 Fuzzy Slippers. I'm prepping some holiday gift baskets."
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile Scarves, the one that can be worn as a scarf, shawl, or headwrap. Thanks!"
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorro de punto grueso. ¿De qué material está hecho? ¿Es lo suficientemente cálido para usar en invierno? Gracias de antemano."
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901 Retro Sunglasses. Thanks!"


In [8]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1xxYNgXaA7CBYKH98OLm9hiHrNrxaoHBminiCB-OILxg


# Task 1. Classify emails

In [5]:
!pip install openai pandas gspread gspread_dataframe


In [54]:
# Import necessary libraries
import pandas as pd         # Used for handling data in DataFrames
import gspread              # Used to interact with Google Sheets
from gspread_dataframe import set_with_dataframe  # Helper function to write DataFrames to Google Sheets
from openai import OpenAI   # OpenAI API client for AI-powered text processing

In [10]:
# Authenticate google sheets
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [11]:
# Set up OpenAI Key
client = OpenAI(
    base_url="https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/",
    api_key="a0BIj000001L8cQMAS"
)

In [12]:
# Define Function to Read Data from Google Sheets
def read_data_frame(document_id, sheet_name):
    """ Reads a Google Sheets document into a pandas DataFrame """
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

In [13]:
# Load the data
document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'  # Replace with your actual document ID
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first few rows for verification
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,"Transport yourself back in time with our retro sunglasses. These vintage-inspired shades offer a cool, nostalgic vibe while protecting your eyes from the sun's rays. Perfect for beach days or city strolls.",1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,"Keep your essentials organized and secure with our sleek wallet. Featuring multiple card slots and a billfold compartment, this stylish wallet is both functional and fashionable. Perfect for everyday carry.",5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,"Add a touch of versatility to your wardrobe with our versatile scarf. This lightweight, multi-purpose accessory can be worn as a scarf, shawl, or even a headwrap. Perfect for transitional seasons or travel.",6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!"
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insight would be appreciated!"


In [14]:
def classify_email(email_text):
    """ Classifies an email as 'Order Request' or 'Product Inquiry' using OpenAI """
    prompt = f"""
    Categorize the following email simply as 'Product Inquiry' or 'Order Request'. Please do not provide any additional text.

    **Order Request** should be used only when:
    1. A **specific product ID** (e.g., 'RSG8901') and a **clear quantity** (e.g., '2', '3 to 4') are mentioned, with the intention to purchase.
    2. If the quantity is ambiguous (e.g., 'three to four', 'a few', 'all remaining'), classify it as **Order Request**. Even if the quantity is not exact, an intention to **order** should be considered as **Order Request**.
    3. If the email expresses an **intent to purchase** a product, even without specifying the exact quantity (e.g., 'I want to order...', 'I'd like to order...', 'Please send me...', 'Can I order...'), it should be classified as **Order Request**. Any phrasing where the customer expresses clear action to **purchase** the product qualifies as an order.

    **Product Inquiry** should be used when:
    1. The email discusses products in a **general** way (e.g., 'I'm looking for a new bag' or 'Do you have any backpacks?').
    2. The email asks for **additional information** or **advice** about a product but does not mention a quantity or clear order intent (e.g., 'What are the differences between these two products?').
    3. The email mentions a **specific product** but lacks a **clear intention to purchase** or **quantity** (e.g., 'I'm interested in the LTH1098 Leather Backpack but not sure about the pockets').
    4. If the email uses phrases like **'I'm looking to buy'**, **'I want to buy'**, or **'Can you tell me more about'** without committing to a specific order or quantity, classify it as **Product Inquiry** rather than an order request.

    **Special Case**:
    If the email contains a **clear request for quantity and a desire to buy**, but is phrased more like a product inquiry (e.g., "I want 5 of these, can you send them to me?"), classify it as **Order Request**.

    Email: {email_text}
    Response:
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [15]:
# Apply email classification

emails_df["category"] = emails_df["message"].apply(classify_email)

In [16]:
# Debug classification
pd.set_option("display.max_colwidth", None)  # Show full text in DataFrame columns
for message in emails_df["message"].head(5):
    print("Email Message:", message)
    print("Predicted Category:", classify_email(message))
    print("-" * 50)

Email Message: Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!
Predicted Category: Order Request
--------------------------------------------------
Email Message: Good morning, I'm looking to buy the VBT2345 Vibrant Tote bag. My name is Jessica and I love tote bags, they're so convenient for carrying all my stuff. Last summer I bought this really cute straw tote that I used at the beach. Oh, and a few years ago I got this nylon tote as a free gift with purchase that I still use for groceries.
Predicted Category: Product Inquiry
--------------------------------------------------
Email Message: Hello, I need a new bag to carry my laptop and documents for work. My name is David and I'm having a hard time deciding which would be better - the LTH1098 Leather Backpack or the Leather Tote? Does one have more organizational pockets than the other? Any insig

In [17]:
# Save Classification to google sheets

output_document = gc.create("AI Automated Email Responses")  # Create output spreadsheet

email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([["email ID", "category"]], "A1:B1")  # Set headers

set_with_dataframe(email_classification_sheet, emails_df[["email_id", "category"]])

print("Task 1: Email classification completed!")

Task 1: Email classification completed!


In [ ]:
# Shareable Link
print(f"Spreadsheet created: https://docs.google.com/spreadsheets/d/{output_document.id}")


# Task 2. Process order requests

In [18]:
def extract_order_details(email_text):
    """ Extracts product IDs and quantities from order request emails using GPT-4o with TSV format. """

    prompt = f"""
    You are an AI order processor. Extract all product IDs and their corresponding quantities from the following email.

    **Email:**
    '{email_text}'

    **Instructions:**
    - Identify each **product ID** mentioned in the email.
    - If the customer specifies a quantity, extract it normally. If no quantity is specified, default it to 1.
    - If the customer asks for **all available stock**, return `all`.
    - If the product or quantity is unclear, return `N/A\t0`.
    - If the product is described generically (e.g., 'leather briefcase' or 'summer sandals') and no product ID is mentioned, infer a general product ID based on the description.
    - Extract all product orders and quantities, even if they are mentioned in different parts of the email.

    **Response Format (Tab-Separated Values):**
    ```
    product_id\tquantity
    RSG8901\t2
    RSG8902\t3
    ```
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    response_text = response.choices[0].message.content.strip()

    # Convert TSV response to list of tuples
    extracted_orders = []
    lines = response_text.split("\n")

    for line in lines[1:]:  # Skip header line
        parts = line.split("\t")
        if len(parts) == 2:
            product_id = parts[0].strip()
            quantity = parts[1].strip()

            # Convert quantity to int unless it's "all"
            quantity = int(quantity) if quantity.isdigit() else quantity

            extracted_orders.append((product_id, quantity))

    return extracted_orders  # Returns list of tuples

In [46]:
def process_orders(emails_df, products_df):
    """
    Processes order requests, checks stock availability, updates stock, and creates order records.
    """

    order_status_records = []

    for _, email in emails_df.iterrows():
        if email["category"] == "Order Request":
            extracted_orders = extract_order_details(email["message"])  # Now returns a list of tuples

            for product_id, quantity in extracted_orders:

                # ✅ Mark invalid orders instead of skipping
                if product_id == "N/A" or quantity in [0, "N/A"]:
                    order_status_records.append({
                        "email_id": email["email_id"],
                        "product_id": "N/A",
                        "quantity": "N/A",
                        "status": "invalid order"
                    })
                    continue

                # Get product stock
                product_row = products_df.loc[products_df["product_id"] == product_id]

                if product_row.empty:
                    status = "out of stock"
                else:
                    available_stock = int(product_row.iloc[0]["stock"])  # Convert stock to integer

                    # ✅ **New Check: If stock is zero, mark as out of stock immediately**
                    if available_stock == 0:
                        status = "out of stock"
                        quantity = 0  # No quantity is being fulfilled
                    else:
                        # Assign all available stock if requested
                        if quantity == "all":
                            quantity = available_stock
                        else:
                            try:
                                quantity = int(quantity)  # Convert quantity safely
                            except ValueError:
                                print(f"Invalid quantity '{quantity}' for product {product_id}. Skipping...")
                                order_status_records.append({
                                    "email_id": email["email_id"],
                                    "product_id": products_df["product_id"],
                                    "quantity": "error",
                                    "status": "invalid order"
                                })
                                continue  # Skip this order if the quantity is invalid

                        if available_stock >= quantity:
                            status = "created"
                            # Update stock safely
                            products_df.loc[products_df["product_id"] == product_id, "stock"] -= quantity
                        else:
                            status = "out of stock"

                # Store order status record
                order_status_records.append({
                    "email_id": email["email_id"],
                    "product_id": product_id,
                    "quantity": quantity,
                    "status": status
                })

    return pd.DataFrame(order_status_records)


In [47]:
import gspread
from gspread_dataframe import set_with_dataframe

def save_orders_to_google_sheets(output_document, order_status_df):
    """
    Saves the processed order status DataFrame to a Google Sheets worksheet.
    If the worksheet already exists, it updates the existing sheet; otherwise, it creates a new one.
    """

    sheet_name = "order-status"

    try:
        # Try to access the existing worksheet
        order_status_sheet = output_document.worksheet(sheet_name)
        order_status_sheet.clear()  # Optional: Clear previous data if needed
        print(f"✅ Worksheet '{sheet_name}' found. Updating existing sheet.")

    except gspread.exceptions.WorksheetNotFound:
        # Create a new worksheet if it doesn’t exist
        order_status_sheet = output_document.add_worksheet(title=sheet_name, rows=50, cols=4)
        print(f"📄 Worksheet '{sheet_name}' not found. Creating a new sheet.")

    # Convert DataFrame to TSV format (tab-separated values)
    tsv_data = order_status_df.to_csv(sep="\t", index=False, header=False)

    # Split TSV into rows for Google Sheets (list of lists)
    rows = [row.split("\t") for row in tsv_data.split("\n") if row]

    # Add headers manually
    headers = [["email ID", "product ID", "quantity", "status"]]
    order_status_sheet.update(headers + rows, "A1")

    print("✅ Task 2: Order processing completed and saved to Google Sheets!")


In [48]:
order_status_df = process_orders(emails_df, products_df)
save_orders_to_google_sheets(output_document, order_status_df)

Processing order: product_id=CLF2109, quantity=5, available_stock=2
Processing order: product_id=CGN2345, quantity=5, available_stock=2
✅ Worksheet 'order-status' found. Updating existing sheet.
✅ Task 2: Order processing completed and saved to Google Sheets!


In [49]:
import pandas as pd

def generate_order_responses(order_status_df, products_df):
    """
    Generates response emails based on order processing results, including invalid orders.
    """

    responses = []

    # Group orders by email ID
    grouped_orders = order_status_df.groupby("email_id")

    for email_id, orders in grouped_orders:
        all_items = []
        out_of_stock_items = []
        invalid_items = []

        for _, order in orders.iterrows():
            product_id = order["product_id"]
            quantity = order["quantity"]
            status = order["status"]

            # Find product name if available
            product_name = products_df.loc[products_df['product_id'] == product_id, 'name'].values
            product_name = product_name[0] if len(product_name) > 0 else product_id

            if status == "created":
                all_items.append(f"{quantity}x {product_name} (Product ID: {product_id})")
            elif status == "out of stock":
                out_of_stock_items.append(f"{quantity}x {product_name} (Product ID: {product_id})")
            elif status == "invalid order":
                invalid_items.append(f"Product: {product_name}, Quantity: {quantity}")

        # Construct the email response
        response = f"Dear Customer,\n\n"

        if invalid_items:
            response += (
                "We were unable to process some items in your order due to missing or unclear information:\n\n"
                + "\n".join(invalid_items)
                + "\n\nPlease provide a valid product ID and quantity so we can assist you further.\n\n"
            )

        if all_items:
            response += (
                "The following items have been successfully processed:\n\n"
                + "\n".join(all_items)
                + "\n\n"
            )

        if out_of_stock_items:
            response += (
                "Unfortunately, the following items are currently out of stock:\n\n"
                + "\n".join(out_of_stock_items)
                + "\n\nWould you like to wait for restock or select an alternative product?\n\n"
            )

        response += "Best regards,\nCustomer Support Team"

        responses.append({"email_id": email_id, "response": response})

    return pd.DataFrame(responses)


In [50]:
# Generate response emails
order_response_df = generate_order_responses(order_status_df, products_df)

# Save to Google Sheets
sheet_name = "order-response"

try:
    response_sheet = output_document.worksheet(sheet_name)
    response_sheet.clear()  # Optional: Clear previous responses if needed
    print(f"Worksheet '{sheet_name}' already exists. Updating existing sheet.")
except gspread.exceptions.WorksheetNotFound:
    response_sheet = output_document.add_worksheet(title=sheet_name, rows=50, cols=2)
    print(f"Worksheet '{sheet_name}' created.")

# Update sheet with headers and data
response_sheet.update([['email_id', 'response']], "A1:B1")
set_with_dataframe(response_sheet, order_response_df)

print("Task 3: Order response emails generated and saved.")


Worksheet 'order-response' already exists. Updating existing sheet.
Task 3: Order response emails generated and saved.


In [51]:
# Shareable Link
print(f"Spreadsheet created: https://docs.google.com/spreadsheets/d/{output_document.id}")

Spreadsheet created: https://docs.google.com/spreadsheets/d/12oyM9_4hYdKiNs04WuNtdW6hGWtNCWtw97harAK2faY


# Task 3. Handle product inquiry

In [52]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import re

def extract_product_mentions(email_text, products_df):
    """
    Extracts product mentions from email text by checking for product IDs or names.
    Uses a case-insensitive search.
    """
    mentioned_products = set()

    for _, row in products_df.iterrows():
        product_id = row["product_id"]
        product_name = row["name"]

        # Match product ID or name in email text
        if re.search(rf"\b{re.escape(product_id)}\b", email_text, re.IGNORECASE) or \
           re.search(rf"\b{re.escape(product_name)}\b", email_text, re.IGNORECASE):
            mentioned_products.add(product_id)

    return list(mentioned_products)

def generate_inquiry_responses(emails_df, products_df):
    """
    Generates responses for product inquiries using relevant product information.
    Populates an inquiry-response sheet with: email ID, response.
    """
    inquiry_responses = []

    for _, email in emails_df.iterrows():
        email_id = email["email_id"]
        email_text = email["message"]

        # Skip non-product-inquiry emails
        if email.get("category", "") != "Product Inquiry":
            continue

        # Extract mentioned products
        mentioned_products = extract_product_mentions(email_text, products_df)

        # Generate a response
        if not mentioned_products:
            response = (f"Dear Customer,\n\nThank you for reaching out! "
                        f"We'd be happy to help you find the perfect product. "
                        f"Could you share a bit more about what you're looking for?\n\n"
                        f"Best,\n[Your Store Name]")
        else:
            product_details = []
            for product_id in mentioned_products:
                product_info = products_df.loc[products_df["product_id"] == product_id]

                if not product_info.empty:
                    product_name = product_info.iloc[0]["name"]
                    price = product_info.iloc[0]["price"]
                    seasons = product_info.iloc[0]["seasons"]
                    description = product_info.iloc[0]["description"]

                    product_details.append(
                        f"- **{product_name}** (${price}) - {description} "
                        f"Great for {seasons.lower()}."
                    )

            product_info_text = "\n".join(product_details)
            response = (f"Dear Customer,\n\nThank you for your inquiry! "
                        f"Here’s some information about the products you mentioned:\n\n"
                        f"{product_info_text}\n\n"
                        f"Let us know if you have any other questions!\n\n"
                        f"Best,\n[Your Store Name]")

        # Store response
        inquiry_responses.append({"email_id": email_id, "response": response})

    return pd.DataFrame(inquiry_responses)

def save_responses_to_google_sheets(output_document, responses_df):
    """
    Saves the inquiry responses to a Google Sheet.
    If the sheet doesn't exist, it creates it.
    """

    sheet_name = "inquiry-response"

    try:
        response_sheet = output_document.worksheet(sheet_name)
        response_sheet.clear()  # Optional: Clear previous responses if needed
        print(f"Worksheet '{sheet_name}' already exists. Updating existing sheet.")
    except gspread.exceptions.WorksheetNotFound:
        response_sheet = output_document.add_worksheet(title=sheet_name, rows=50, cols=2)
        print(f"Worksheet '{sheet_name}' created.")

    # Update sheet with headers and data
    response_sheet.update([['email_id', 'response']], "A1:B1")
    set_with_dataframe(response_sheet, responses_df)

    print("✅ Task 3: Product inquiry responses generated and saved.")

# Example usage
inquiry_responses_df = generate_inquiry_responses(emails_df, products_df)
save_responses_to_google_sheets(output_document, inquiry_responses_df)


Worksheet 'inquiry-response' created.
✅ Task 3: Product inquiry responses generated and saved.


In [ ]:
# Shareable Link
print(f"Spreadsheet created: https://docs.google.com/spreadsheets/d/{output_document.id}")